In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("restaurant-review-average")
sc = SparkContext(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/02 04:08:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


csv 파일을 RDD 로딩

In [3]:
filepath = '/home/ubuntu/working/spark-examples/data/restaurant_reviews.csv'

In [4]:
lines = sc.textFile(f'file:///{filepath}')
lines

file:////home/ubuntu/working/spark-examples/data/restaurant_reviews.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [5]:
# collect(): rdd 내의 데이터를 모두 출력
lines.collect()

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [6]:
# header 제거
header = lines.first()

datas = lines.filter(lambda row : row != header)
datas

# 불변성 : 제거, 변경 등이 안됨
# 이거 빼고 불러오는식으로 제거를 구현

PythonRDD[3] at RDD at PythonRDD.scala:53

In [7]:
datas.collect()

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [9]:
datas.collect()[0]
# 문자열 
# 문자열을 내가 원하는 형태로 바꿔주는 행위 : parsing

'0,짜장면,중식,125,'

카테고리 별 리뷰 개수 평균 구하기

In [11]:
# 2,3 번 필드 가져오기

# parsing path 지정
def parse_task(row):
    # 2번, 3번 필드 반환. 단, 3번 필드는 정수로 리턴
    fields = row.split(',')
    
    category = fields[2]
    review_cnt = int(fields[3])
    
    return category, review_cnt

In [12]:
parse_task('0,짜장면,중식,125,')

('중식', 125)

RDD 내의 모든 데이터에 대해 `parse_task`함수를 적용 후 추출(map)

In [ ]:
# RDD(datas)에 task 적용 시킨 새로운 RDD 만들기 

In [13]:
category_reviews = datas.map(parse_task)
category_reviews

PythonRDD[4] at RDD at PythonRDD.scala:53

In [14]:
category_reviews.collect() #key-Value RDD

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [15]:
# 카테고리 별 리뷰 개수 평균

# 데이터 개수 구하기
# value에 값을 하나 더 (변환(T)가 한번 더)
# '중식',(125,1)
#  key , value

# value에 대한 변환 : mapValues
category_review_count = category_reviews.mapValues(lambda x : (x, 1))
category_review_count.collect()

# 파티션(key) 변경없이 value에서만 변환이 일어남
# key_value RDD 에서는 value 만 변경하는 것이 좋음

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

reduceByKey를 이용해 `Key`별 `Value`에 대한 집계를 수행. 집계 Task 정의가 필요하다(함수)

In [17]:
reduced = category_review_count.reduceByKey(lambda x, y : (x[0] + y[0], x[1] + y[1]))
reduced.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [ ]:
#x : 누적값, y : 새로운 값
#첫번째 중식에 대한 Value가 x에 들어감
#키 별로 연산함 그래서 reduceByKey

In [18]:
average = reduced.mapValues(lambda x : x[0] / x[1])
average.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]

In [19]:
sc.stop()